In [54]:
# import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import inspect
import time
import pickle

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import src.simulation as s
import src.utils as utils

%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
# Read input variables
input_args = utils.load_input_args('../input/simulation.json')

In [56]:
# instantiate simulation class

simulation =  s.Simulation()

/Users/bojansimoski/dev/eur/ABM-PA/src/simulation.py:24: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  #         self.CommunicationDataPopulation = p.CommunicationDataPopulation('Communication data population', self.input_args)
/Users/bojansimoski/dev/eur/ABM-PA/src/simulation.py:25: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  #         self.model = m.DiffusionModel(self.input_args)
<ipython-input-56-82f461c16b09>:3: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  simulation =  s.Simulation()
/Users/bojansimoski/dev/eur/ABM-PA/src/simulation.py:26: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,

### Testing the new population - to remove after done

In [57]:
df_network, df_ego = simulation.PeerNominatedDataPopulation.population_network_summary(simulation.PeerNominatedDataPopulation.graph)

/Users/bojansimoski/anaconda3/envs/ABM-PA/lib/python3.9/site-packages/networkx/algorithms/assortativity/correlation.py:282: RuntimeWarning: invalid value encountered in double_scalars
  return (xy * (M - ab)).sum() / np.sqrt(vara * varb)


In [58]:
df_network.head()

,SchoolClassID,NumberOfAgents,PercentageFemale,NumberConnections,Density,IsolatedNodes,CentralizationInDegree,CentralizationOutDegree,CentralizationCloseness,CentralizationBetweenness,DegreeAssortativity,AverageEnvironmentScore,AverageBMIScore
0,78.0,18,55,114,0.37,0,0.43,0.68,-0.38,-0.52,-0.55,0.59,-1.0
1,81.0,19,52,184,0.54,0,0.29,0.52,-0.47,-0.12,0.04,0.48,-1.0
2,100.0,19,36,171,0.50,0,0.35,0.56,-0.66,-0.27,-0.33,0.56,-1.0
3,124.0,17,64,181,0.67,0,0.27,0.38,-1.21,-0.12,-0.23,0.53,-1.0
4,125.0,16,56,132,0.55,0,0.42,0.51,-1.71,-0.21,-0.04,0.50,-1.0


In [59]:
df_ego.head()

,ParticipantID,SchoolClassID,Gender,PA,BMI,Environment,InDegree,OutDegree,Closeness,Betweenness,Average_Weight
18,1605,81.0,1,0.908846,-1.0,0.583333,8,18,0.620690,0.043321,0.319444
19,1606,81.0,1,1.316720,-1.0,0.488193,13,13,0.782609,0.056189,0.384615
20,1607,81.0,0,1.071277,-1.0,0.333333,7,8,0.620690,0.016476,0.343750
21,1608,81.0,1,1.098937,-1.0,0.916667,10,18,0.692308,0.042131,0.513889
22,1609,81.0,1,0.528888,-1.0,0.333333,8,7,0.620690,0.002682,0.392857


In [60]:
df_network_social, df_ego_social = simulation.SocialFacilitationPopulation.population_network_summary(simulation.SocialFacilitationPopulation.graph)

In [62]:
df_network_social.head()

,SchoolClassID,NumberOfAgents,PercentageFemale,NumberConnections,Density,IsolatedNodes,CentralizationInDegree,CentralizationOutDegree,CentralizationCloseness,CentralizationBetweenness,DegreeAssortativity,AverageEnvironmentScore,AverageBMIScore
0,78.0,18,55,27,0.09,3,0.88,0.93,-0.25,-0.07,-0.24,0.59,-1.0
1,81.0,19,52,67,0.20,0,0.61,0.72,-0.38,-1.21,-0.26,0.48,-1.0
2,100.0,19,36,76,0.22,2,0.53,0.66,-0.28,-0.64,-0.03,0.56,-1.0
3,124.0,17,64,94,0.35,0,0.62,0.62,-1.28,-0.76,-0.17,0.53,-1.0
4,125.0,16,56,51,0.21,1,0.68,0.73,-0.53,-0.37,-0.06,0.50,-1.0


In [63]:
df_ego_social.head()

,ParticipantID,SchoolClassID,Gender,PA,BMI,Environment,InDegree,OutDegree,Closeness,Betweenness,Average_Weight
18,1605,81.0,1,0.908846,-1.0,0.583333,5,2,0.334491,0.032771,0.666667
19,1606,81.0,1,1.316720,-1.0,0.488193,6,6,0.501736,0.489615,0.388889
20,1607,81.0,0,1.071277,-1.0,0.333333,1,2,0.247009,0.000000,0.333333
21,1608,81.0,1,1.098937,-1.0,0.916667,2,9,0.314815,0.045207,0.703704
22,1609,81.0,1,0.528888,-1.0,0.333333,2,3,0.263206,0.001906,0.444444


# 1. Peer-Nomination Network Data

In [18]:
# Read tuned parameter combinations (See tuning_nom_network.ipynb)
pars_nomination = pd.read_csv('../output/opt_pars_nomination.csv', sep=',', header=0)
pars_nomination

,threshold,ipa,error
0,0.0125,0.0075,0.188295
1,0.0150,0.0050,0.192803
2,0.0125,0.0050,0.193931
3,0.0125,0.0025,0.195202
4,0.0150,0.0025,0.196098
...,...,...,...
95,0.0450,0.0100,0.341865
96,0.0350,0.0175,0.342947
97,0.0500,0.0025,0.343016
98,0.0475,0.0075,0.343105


### 1.1. Pre-Intervention simulations

In [19]:
# Run pre-intervention for each parameter set
list_pre_results_child_nom = []
list_pre_results_class_nom = []
list_snapshot_pa_nom = []

for index, row in pars_nomination.iterrows():
    start = time.time()
    # run pre-intervention
    pre_results_nom, pre_results_avg_nom, snapshot_pa_nom = simulation.simulate_preinterventions(700, 'nomination', row['threshold'], row['ipa'])

    list_pre_results_child_nom.append(pre_results_nom)
    list_pre_results_class_nom.append(pre_results_avg_nom)
    list_snapshot_pa_nom.append(snapshot_pa_nom)

    end = time.time()
    print(index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

# Save model output to object
file = open('../output/simulation/nomination/list_pre_results_child.obj', 'wb')
pickle.dump(list_pre_results_child_nom, file)

file = open('../output/simulation/nomination/list_pre_results_class.obj', 'wb')
pickle.dump(list_pre_results_class_nom, file)

file = open('../output/simulation/nomination/list_snapshot_pa.obj', 'wb')
pickle.dump(list_snapshot_pa_nom, file)


0 0.0125 0.0075 Time elapsed: 8.252204179763794  seconds
1 0.015 0.005 Time elapsed: 8.186835050582886  seconds
2 0.0125 0.005 Time elapsed: 8.17768120765686  seconds
3 0.0125 0.0025 Time elapsed: 8.192239046096802  seconds
4 0.015 0.0025 Time elapsed: 8.23025894165039  seconds
5 0.015 0.0075 Time elapsed: 8.191330671310425  seconds
6 0.0125 0.01 Time elapsed: 8.227228164672852  seconds
7 0.01 0.005 Time elapsed: 8.165056943893433  seconds
8 0.01 0.0075 Time elapsed: 8.119465112686157  seconds
9 0.017499999999999998 0.0025 Time elapsed: 8.15949010848999  seconds
10 0.015 0.01 Time elapsed: 8.120134830474854  seconds
11 0.017499999999999998 0.0075 Time elapsed: 8.67418885231018  seconds
12 0.01 0.0025 Time elapsed: 8.266731977462769  seconds
13 0.017499999999999998 0.005 Time elapsed: 8.228315353393555  seconds
14 0.017499999999999998 0.01 Time elapsed: 8.603729009628296  seconds
15 0.02 0.0025 Time elapsed: 8.241947174072266  seconds
16 0.02 0.005 Time elapsed: 8.422947883605957  secon

### 1.2. Intervention Simulations


In [20]:
# Run interventions for each parameter combination
list_results_child_nom = []
list_results_class_nom = []
list_influential_agents_nom = []

for index, row in pars_nomination.iterrows():
    start = time.time()
    # run intervention
    results_nom, results_avg_nom, agents_per_intervention_nom = simulation.simulate_interventions(700, 'nomination', list_snapshot_pa_nom[index], row['threshold'], row['ipa'], index)

    list_results_child_nom.append(results_nom)
    list_results_class_nom.append(results_avg_nom)
    list_influential_agents_nom.append(agents_per_intervention_nom)

    end = time.time()
    print(index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

# Save model output to object
file = open('../output/simulation/nomination/list_results_child.obj', 'wb')
pickle.dump(list_results_child_nom, file)

file = open('../output/simulation/nomination/list_results_class.obj', 'wb')
pickle.dump(list_results_class_nom, file)

file = open('../output/simulation/nomination/list_influential_agents.obj', 'wb')
pickle.dump(list_influential_agents_nom, file)


0 0.0125 0.0075 Time elapsed: 54.42180681228638  seconds
1 0.015 0.005 Time elapsed: 53.873947858810425  seconds
2 0.0125 0.005 Time elapsed: 53.252638816833496  seconds
3 0.0125 0.0025 Time elapsed: 48.45851993560791  seconds
4 0.015 0.0025 Time elapsed: 51.74261212348938  seconds
5 0.015 0.0075 Time elapsed: 52.130955934524536  seconds
6 0.0125 0.01 Time elapsed: 51.124966859817505  seconds
7 0.01 0.005 Time elapsed: 57.365662813186646  seconds
8 0.01 0.0075 Time elapsed: 48.68152403831482  seconds
9 0.017499999999999998 0.0025 Time elapsed: 49.44387722015381  seconds
10 0.015 0.01 Time elapsed: 47.585808992385864  seconds
11 0.017499999999999998 0.0075 Time elapsed: 51.352731227874756  seconds
12 0.01 0.0025 Time elapsed: 51.10161304473877  seconds
13 0.017499999999999998 0.005 Time elapsed: 49.62642192840576  seconds
14 0.017499999999999998 0.01 Time elapsed: 49.29880714416504  seconds
15 0.02 0.0025 Time elapsed: 47.44559693336487  seconds
16 0.02 0.005 Time elapsed: 47.1871440410

# 2. Online-Communication Network Data


In [21]:
# Read tuned parameter combinations (See tuning_com_network.ipynb)
pars_communication = pd.read_csv('../output/opt_pars_communication.csv', sep=',', header=0, encoding='latin-1')
pars_communication

,threshold,ipa,error
0,0.0050,0.0050,0.363408
1,0.0075,0.0025,0.383596
2,0.0050,0.0025,0.389370
3,0.0075,0.0050,0.410030
4,0.0100,0.0025,0.413637
...,...,...,...
95,0.0900,0.0050,0.561751
96,0.0425,0.0125,0.562553
97,0.0750,0.0025,0.562816
98,0.0500,0.0125,0.562944


### 2.1. Pre-Intervention simulations

In [22]:
# Run pre-intervention for each parameter combination
list_pre_results_child_com = []
list_pre_results_class_com = []
list_snapshot_pa_com = []

for index, row in pars_communication.iterrows():
    start = time.time()
    # run pre-intervention
    pre_results_com, pre_results_avg_com, snapshot_pa_com = simulation.simulate_preinterventions(700,'communication',row['threshold'],row['ipa'])

    list_pre_results_child_com.append(pre_results_com)
    list_pre_results_class_com.append(pre_results_avg_com)
    list_snapshot_pa_com.append(snapshot_pa_com)

    end = time.time()
    print(index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

# Save model output to object
file = open('../output/simulation/communication/list_pre_results_child.obj', 'wb')
pickle.dump(list_pre_results_child_com, file)

file = open('../output/simulation/communication/list_pre_results_class.obj', 'wb')
pickle.dump(list_pre_results_class_com, file)

file = open('../output/simulation/communication/list_snapshot_pa.obj', 'wb')
pickle.dump(list_snapshot_pa_com, file)


0 0.005 0.005 Time elapsed: 6.691161870956421  seconds
1 0.0075 0.0025 Time elapsed: 6.716557025909424  seconds
2 0.005 0.0025 Time elapsed: 6.627782106399536  seconds
3 0.0075 0.005 Time elapsed: 6.883543968200684  seconds
4 0.01 0.0025 Time elapsed: 6.76504111289978  seconds
5 0.01 0.005 Time elapsed: 6.53136420249939  seconds
6 0.0125 0.0025 Time elapsed: 6.594964027404785  seconds
7 0.01 0.0075 Time elapsed: 6.4747161865234375  seconds
8 0.0325 0.0025 Time elapsed: 7.0328710079193115  seconds
9 0.035 0.0025 Time elapsed: 7.379678010940552  seconds
10 0.03 0.0025 Time elapsed: 7.192678928375244  seconds
11 0.0275 0.0025 Time elapsed: 6.920289039611816  seconds
12 0.037500000000000006 0.0025 Time elapsed: 6.939225673675537  seconds
13 0.04 0.0025 Time elapsed: 6.978071212768555  seconds
14 0.0425 0.0025 Time elapsed: 7.235051155090332  seconds
15 0.0125 0.005 Time elapsed: 7.360620021820068  seconds
16 0.015 0.0025 Time elapsed: 6.827073097229004  seconds
17 0.045000000000000005 0.00

In [23]:
# Run interventions for each parameter combination
list_results_child_com = []
list_results_class_com = []
list_influential_agents_com = []

for index, row in pars_communication.iterrows():
    start = time.time()
    # run intervention
    results_com, results_avg_com, agents_per_intervention_com = simulation.simulate_interventions(700,'communication',list_snapshot_pa_com[index],row['threshold'],row['ipa'],index)

    list_results_child_com.append(results_com)
    list_results_class_com.append(results_avg_com)
    list_influential_agents_com.append(agents_per_intervention_com)

    end = time.time()
    print(index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

# Save model output to object
file = open('../output/simulation/communication/list_results_child.obj', 'wb')
pickle.dump(list_results_child_com, file)

file = open('../output/simulation/communication/list_results_class.obj', 'wb')
pickle.dump(list_results_class_com, file)

file = open('../output/simulation/communication/list_influential_agents.obj', 'wb')
pickle.dump(list_influential_agents_com, file)


0 0.005 0.005 Time elapsed: 44.01352119445801  seconds
1 0.0075 0.0025 Time elapsed: 40.928443908691406  seconds
2 0.005 0.0025 Time elapsed: 41.01640605926514  seconds
3 0.0075 0.005 Time elapsed: 41.98280692100525  seconds
4 0.01 0.0025 Time elapsed: 42.27522373199463  seconds
5 0.01 0.005 Time elapsed: 41.84689378738403  seconds
6 0.0125 0.0025 Time elapsed: 41.60752201080322  seconds
7 0.01 0.0075 Time elapsed: 41.404396057128906  seconds
8 0.0325 0.0025 Time elapsed: 42.16863393783569  seconds
9 0.035 0.0025 Time elapsed: 42.27508091926575  seconds
10 0.03 0.0025 Time elapsed: 45.14508390426636  seconds
11 0.0275 0.0025 Time elapsed: 45.881263732910156  seconds
12 0.037500000000000006 0.0025 Time elapsed: 43.12951302528381  seconds
13 0.04 0.0025 Time elapsed: 42.656561851501465  seconds
14 0.0425 0.0025 Time elapsed: 41.67401695251465  seconds
15 0.0125 0.005 Time elapsed: 41.9348669052124  seconds
16 0.015 0.0025 Time elapsed: 41.7665638923645  seconds
17 0.045000000000000005 0.